# M/G/1 model

- simulator seems running, but prior in this case is non-standard uniform, so proposal currently generates outside prior bounds !

In [ ]:
%matplotlib inline

import sys
sys.path.append('/home/mackelab/Desktop/Projects/Biophysicality/code/snl_py3/') # SNL does not have a setup.py

In [ ]:
cd '/home/mackelab/Desktop/Projects/Biophysicality/code/snl_py3'

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import simulators.mg1 as sim

import delfi.generator as dg
import delfi.distribution as dd
from delfi.utils.viz import plot_pdf
import delfi.inference as infer

seed = 42

# SNPE parameters
    
# training schedule
n_train=3000
n_rounds=5

# fitting setup
minibatch=100
epochs=50

# network setup
n_hiddens=[50]
reg_lambda=0.01

# convenience
pilot_samples=0
svi=False
verbose=True
prior_norm=False


In [ ]:
from delfi.simulator import BaseSimulator

# prior 

#prior = sim.Prior()
prior = dd.Uniform(lower=[ 0, 0,  0  ], 
                   upper=[10,20,1./3.])

# model 
model_snl = sim.Model()
class MG1(BaseSimulator):
    """M/G/1 simulator

    Parameters
    ----------
    dim : int
        Number of dimensions of parameters
    seed : int or None
        If set, randomness is seeded
    """        

    def gen_single(self, params):
        """ params = (lower bound of server processing time,
                      upper bound of server processing time,
                      rate customer arrivals )
        
        """
        return model_snl.sim(params)
        
model = MG1(dim_param=3)

# summary statistics
summary = sim.Stats()

# generator
g = dg.Default(prior=prior, model=model, summary=summary, seed=seed+41)

In [ ]:
pars_true = np.array([1, 5, 0.2])  # taken from SNL paper
obs = g.model.gen_single(pars_true)  # should also recover
obs_stats = g.summary.calc([obs])    # xo from SNL paper !

In [ ]:
inf = infer.SNPEC(generator=g, obs=obs_stats, prior_norm=prior_norm, init_norm=False,
                 pilot_samples=pilot_samples, seed=seed, reg_lambda=reg_lambda, svi=svi,
                 n_components=1, n_hiddens=n_hiddens, verbose=True)

log, trn_data, posteriors = inf.run(n_train=n_train, epochs=epochs, minibatch=minibatch, n_rounds=n_rounds,  
                   moo='p_tilda')

In [ ]:
fig,_=plot_pdf(posteriors[0], 
               lims=[[0,10], [0,20], [0,0.34]],
               gt=pars_true, 
               samples=trn_data[0][0].T,
               resolution=100,
               ticks=True,
               figsize=(16,16));

In [ ]:
%autoreload